In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
knn_clf.score(X_test, y_test)

0.9888888888888889

### 寻找最好的k

In [5]:
best_score = 0.0
best_k = -1
for k in range (1,11):
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(X_train,y_train)
    score = knn_clf.score(X_test,y_test)
    if (score > best_score):
        best_k = k
        best_score = score

print('best_k = ',best_k)
print('best_score = ',best_score)

best_k =  4
best_score =  0.9916666666666667


In [6]:
# 如果找到最好的值 在边界上 ，应该增加范围

### 考虑距离？不考虑距离

In [7]:
best_score = 0.0
best_k = -1
best_method = ""
for method in ["uniform","distance"]:
    for k in range (1,11):
        knn_clf = KNeighborsClassifier(n_neighbors=k,weights=method)
        knn_clf.fit(X_train,y_train)
        score = knn_clf.score(X_test,y_test)
        if (score > best_score):
            best_k = k
            best_score = score
            best_method = method

print('best_method =',best_method)
print('best_k = ',best_k)
print('best_score = ',best_score)

best_method = uniform
best_k =  4
best_score =  0.9916666666666667


### 搜索明可夫斯基距离相应的p

In [8]:
%%time

best_score = 0.0
best_k = -1
best_p  = -1
for k in range (1,11):
    for p in range(1,6):
        knn_clf = KNeighborsClassifier(n_neighbors=k,weights="distance",p=p)
        knn_clf.fit(X_train,y_train)
        score = knn_clf.score(X_test,y_test)
        if (score > best_score):
            best_k = k
            best_score = score
            best_p = p

print('best_p =',p)
print('best_k = ',best_k)
print('best_score = ',best_score)

best_p = 5
best_k =  3
best_score =  0.9888888888888889
CPU times: user 16.3 s, sys: 201 ms, total: 16.5 s
Wall time: 17.4 s


### grid search

In [9]:
param_grid = [
    {
        "weights":['uniform'],
        "n_neighbors":[i for i in range(1, 11)]
    },
    {
        "weights":['distance'],
        "n_neighbors":[i for i in range(1, 11)],
        "p":[i for i in range(1, 6)]
    }
]


In [10]:
knn_clf = KNeighborsClassifier()

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
grid_search = GridSearchCV(knn_clf,param_grid)

In [13]:
%%time
grid_search.fit(X_train,y_train)

CPU times: user 1min 18s, sys: 1.12 s, total: 1min 20s
Wall time: 1min 25s


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [15]:
grid_search.best_score_

0.9860820751064653

In [17]:
knn_clf = grid_search.best_estimator_


In [19]:
knn_clf.score(X_test,y_test)

0.9833333333333333

In [20]:
%%time
grid_search = GridSearchCV(knn_clf,param_grid,n_jobs=-1)

CPU times: user 39 µs, sys: 0 ns, total: 39 µs
Wall time: 43.9 µs


In [22]:
%%time
grid_search = GridSearchCV(knn_clf,param_grid,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 286 tasks      | elapsed:   25.2s


CPU times: user 658 ms, sys: 74 ms, total: 732 ms
Wall time: 27 s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   27.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=1, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)